In [2]:
import pandas as pd
import numpy as np
import datetime

In [16]:
#load dataset copy 
df_featured = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/inputated_data.csv')

5. Feature engeneering

insert day of the week

In [17]:
#insert new column day of week as integer (1=Monday,...,7=Sunday)   
df_featured['Wochentag'] = pd.to_datetime(df_featured['Datum']).dt.dayofweek + 1
print(df_featured[['Datum', 'Wochentag']].head())


        Datum  Wochentag
0  2013-07-01          1
1  2013-07-01          1
2  2013-07-01          1
3  2013-07-01          1
4  2013-07-01          1


insert holiday

In [18]:
#insert new column holiday from 2013-07-01 to 2018-07-31 
holidays = [
    '2013-10-03', '2013-12-25', '2013-12-26',
    '2014-01-01', '2014-04-18', '2014-04-21',
    '2014-05-01', '2014-05-29', '2014-06-09',
    '2014-10-03', '2014-12-25', '2014-12-26',
    '2015-01-01', '2015-04-03', '2015-04-06',
    '2015-05-01', '2015-05-14', '2015-05-25',
    '2015-10-03', '2015-12-25', '2015-12-26',
    '2016-01-01', '2016-03-25', '2016-03-28',
    '2016-05-01', '2016-05-05', '2016-05-16',
    '2016-10-03', '2016-12-25', '2016-12-26',
    '2017-01-01', '2017-04-14', '2017-04-17',
    '2017-05-01', '2017-05-25', '2017-06-05',
    '2017-10-03', '2017-10-31', '2017-12-25', '2017-12-26',
    '2018-01-01', '2018-03-30', '2018-04-02',
    '2018-05-01', '2018-05-10', '2018-05-21'
]
df_featured['Feiertag'] = df_featured['Datum'].isin(holidays)
#convert boolean to integer (0 not holiday, 1 holiday)
df_featured['Feiertag'] = df_featured['Feiertag'].astype(int)

print(df_featured[['Datum', 'Feiertag']].head())

        Datum  Feiertag
0  2013-07-01         0
1  2013-07-01         0
2  2013-07-01         0
3  2013-07-01         0
4  2013-07-01         0


set season from day and month

In [13]:
# create column season like this: Winter from 22 December to 20 march, Spring from 21 March to 21 june, Summer from 22 June to 21 september, Autumn from 22 September to 21 december
def get_season(date_or_month):
    # If a date-like is passed (string/Timestamp/date), use exact day-month boundaries
    if isinstance(date_or_month, (str, pd.Timestamp, datetime.date)):
        d = pd.to_datetime(date_or_month)
        m, day = d.month, d.day
        if (m == 12 and day >= 22) or m in (1, 2) or (m == 3 and day <= 20):
            return 1 #'Winter'
        if (m == 3 and day >= 21) or m in (4, 5) or (m == 6 and day <= 21):
            return 2 #'Spring'
        if (m == 6 and day >= 22) or m in (7, 8) or (m == 9 and day <= 21):
            return 3 #'Summer'
        return 4 #'Autumn'
    # If an integer month is passed, fall back to month-based mapping
    m = int(date_or_month)
    if m in (12, 1, 2):
        return 1 #'Winter'
    if m in (3, 4, 5):
        return 2 #'Spring'
    if m in (6, 7, 8):
        return 3 #'Summer'
    return 4 #'Autumn'

df_featured['Monat'] = pd.to_datetime(df_featured['Datum']).dt.month
# Derive season based on the full date (day-month boundaries are respected)
df_featured['Jahreszeit'] = pd.to_datetime(df_featured['Datum']).apply(get_season)
print(df_featured[['Datum', 'Monat', 'Jahreszeit']].head())


        Datum  Monat  Jahreszeit
0  2013-07-01      7           3
1  2013-07-01      7           3
2  2013-07-01      7           3
3  2013-07-01      7           3
4  2013-07-01      7           3


In [19]:
# insert new column for school holidays (ferien)
school_holidays = [
    # Summer holidays
    ('2013-07-22', '2013-08-31'),
    ('2014-07-21', '2014-08-31'),
    ('2015-07-20', '2015-08-31'),
    ('2016-07-18', '2016-08-31'),
    ('2017-07-17', '2017-08-31'),
    ('2018-07-16', '2018-08-31'),
    # Autumn holidays
    ('2013-10-14', '2013-10-26'),
    ('2014-10-13', '2014-10-25'),
    ('2015-10-12', '2015-10-24'),
    ('2016-10-10', '2016-10-22'),
    ('2017-10-09', '2017-10-21'),
    ('2018-10-08', '2018-10-20'),
    # Christmas holidays
    ('2013-12-23', '2014-01-05'),
    ('2014-12-22', '2015-01-04'),
    ('2015-12-21', '2016-01-03'),
    ('2016-12-23', '2017-01-06'),
    ('2017-12-22', '2018-01-05'),
    # Winter holidays
    ('2014-02-03', '2014-02-15'),
    ('2015-02-02', '2015-02-14'),
    ('2016-02-01', '2016-02-13'),
    ('2017-01-30', '2017-02-11'),
    ('2018-01-29', '2018-02-10'),
    # Easter holidays
    ('2013-03-25', '2013-04-06'),
    ('2014-04-14', '2014-04-25'),
    ('2015-04-06', '2015-04-18'),
    ('2016-03-21', '2016-04-01'),
    ('2017-04-10', '2017-04-21'),
    ('2018-03-26', '2018-04-06'),
    # Spring holidays
    ('2013-05-20', '2013-05-31'),
    ('2014-05-19', '2014-05-30'),
    ('2015-05-18', '2015-05-29'),
    ('2016-05-16', '2016-05-27'),
    ('2017-05-15', '2017-05-26'),
    ('2018-05-14', '2018-05-25'),
]       
df_featured['Ferien'] = 0
for start, end in school_holidays:
    mask = (pd.to_datetime(df_featured['Datum']) >= pd.to_datetime(start)) & (pd.to_datetime(df_featured['Datum']) <= pd.to_datetime(end))
    df_featured.loc[mask, 'Ferien'] = 1     



In [20]:
#check nans
print(df_featured.isna().sum())

id                        0
Datum                     0
Warengruppe               0
Umsatz                 1897
KielerWoche               0
Bewoelkung                0
Temperatur                0
Windgeschwindigkeit       0
Wettercode                0
Woche                     0
Monat                     0
Wochentag                 0
Feiertag                  0
Ferien                    0
dtype: int64


In [21]:
#save featured dataframe to a new CSV file
df_featured.to_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Processed\featured_data.csv', index=False)